In [ ]:
import cffi

ffi = cffi.FFI()
ffi.cdef("uint64_t my_length(lean_obj_arg);")

lib = ffi.dlopen("liblean.so")


https://github.com/dselsam/lean-python-bindings

https://github.com/leanprover/lean4/tree/master/src/lake/examples/reverse-ffi

https://docs.lean-lang.org/lean4/doc/dev/ffi.html#borrowing

https://github.com/leanprover/lean4/tree/master/tests/compiler/foreign

In [3]:
%%file /tmp/hello.lean

@[export my_length]
def myLength (s : String) : UInt64 :=
  s.length.toUInt64


Overwriting /tmp/hello.lean


In [9]:
! cd /tmp && lean -c /tmp/hello.c  /tmp/hello.lean && leanc  -o /tmp/hello.o /tmp/hello.c

info: Version 4.1.2 of elan is available! Use `elan self update` to update.
info: downloading https://github.com/leanprover/lean4/releases/download/v4.20.1/lean-4.20.0-linux.tar.zst
347.5 MiB / 347.5 MiB (100 %)  10.4 MiB/s ETA:   0 s
info: installing /home/philip/.elan/toolchains/leanprover--lean4---v4.20.1
ld.lld: error: undefined symbol: main
>>> referenced by start.S:104 (../sysdeps/x86_64/start.S:104)
>>>               /home/philip/.elan/toolchains/leanprover--lean4---v4.20.1/lib/Scrt1.o:(_start)
clang: error: linker command failed with exit code 1 (use -v to see invocation)


ModuleNotFoundError: No module named 'leandojo'